# Przetwarzanie Danych Ustrukturyzowanych [PDU]
## Praca domowa nr 4
### Autor: Elżbieta Jowik
#### Termin oddania pracy: 10.06.2019

## 0.1. Importowanie pakietów.

In [1]:
import numpy as np
import pandas as pd
import os
import tempfile
import sqlite3

## 0.2. Wczytanie danych.

In [67]:
# Wczytujemy pliki
# 1. najpierw wylistowujemy zawartość katalogu, w którym znajdują się dane

In [3]:
listing = os.listdir("/home/elzbieta/pdu_R/PD1/travel_stackexchange_com")
listing

['Badges.csv',
 'PostLinks.csv',
 'Posts.csv',
 'Votes.csv',
 'Tags.csv',
 'Comments.csv',
 'Users.csv']

In [4]:
# 2. następnie na pomocą pętli wczytujemy pliki, jednocześnie przypisując im odpowiednie nazwy.

In [4]:
n = len(listing)
names = [None for i in range(n)]

for i in range(n):
    names[i] = listing[i].split(".")[0]
    df = pd.read_csv(listing[i])
    vars()[names[i]] = df

In [5]:
# Tworzymy sobie tymczasową bazę danych jak na laboratoriach
baza = os.path.join(tempfile.mkdtemp(), "baza.db")

if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)
# Wykorzystujemy funkcję to_sql
Badges.to_sql("Badges", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Votes.to_sql("Votes", conn)
Tags.to_sql("Tags", conn)
Comments.to_sql("Comments", conn)
Users.to_sql("Users", conn)

# Zadanie 1.

## 1.1. Jaki rezultat mamy osiągnąć?
### W tym zadaniu chcemy wydobyć informacje o użytkownikach, których posty w ogólności cieszyły się największą popularnością, czyli 10 użytkowników, których posty dostały najwięcej 'gwiazdek'. 
Kryterium oceny popularności będzie stanowić zsumowana wartość zmiennej FavoriteCount dla poszczególnych użytkowników, którą od teraz określać będziemy jako _FavoriteTotal_. 
Dane, na których nam zależy to nazwa wiek i lokalizacja użytkowników, ich napopularniejszy post (_MostFavoriteQuestion_), wartość zmiennej FavoriteCount dla tego postu (_MostFavoriteQuestionLikes_) oraz wartość zmiennej _FavoriteTotal_. 
Uwaga! Typ postów, które będziemy rozważać to 1. 

In [6]:
sql_1 = pd.read_sql_query(
    """
    SELECT
        Users.DisplayName,
        Users.Age,
        Users.Location,
        SUM(Posts.FavoriteCount) AS FavoriteTotal,
        Posts.Title AS MostFavoriteQuestion,
        MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
    FROM Posts
    JOIN Users ON Users.Id=Posts.OwnerUserId
    WHERE Posts.PostTypeId=1
    GROUP BY OwnerUserId
    ORDER BY FavoriteTotal DESC
    LIMIT 10
    """, conn)

In [7]:
# Wybieramy te wiersze, dla których wartości w kolumie PostType wynosi 1, 
df = Posts[Posts.PostTypeId==1]
# grupujemy po zmiennej "OwnerUserId"
tmp = df.groupby('OwnerUserId')
# tuple: (grupa, odpowiadajaca jej DF)
# for grupa, ramka in df:
#    print(ramka)

In [8]:
# W poszczególnych grupach na kolumnie FavoriteCount agregujemy funkcje sum i max
tmp1 =  pd.DataFrame([{'FavoriteTotal' : ramka.FavoriteCount.sum(skipna = True),
                       'OwnerUserId': grupa} 
                        for (grupa, ramka) in tmp])

tmp2 = pd.DataFrame([{'MostFavoriteQuestionLikes': ramka.FavoriteCount.max(skipna = True),
                      'OwnerUserId': grupa} 
                       for (grupa, ramka) in tmp])

In [9]:
# W tym kroku najpierw z ramki Posts wybieramy wiersze dla których PostTypeId == 1 oraz kolumny: 
# 'FavoriteCount', 'Title', 'OwnerUserId'
# Następnie łączymy ją ze stworzoną wyżej ramką, w której mamy maksymalne mawrtości zmiennej
# FavoriteCount 
# Kryteria złączenia to równe Id użytkowników (OwnerUserId) oraz równość zmiennych 
# "MostFavoriteQuestionLikes" i "FavoriteCount"

tmp3 = Posts[Posts.PostTypeId == 1][['FavoriteCount', 'Title', 'OwnerUserId']].merge(tmp2, right_on = [
    "OwnerUserId", "MostFavoriteQuestionLikes"], left_on = ["OwnerUserId", "FavoriteCount"]).drop(
    columns = 'FavoriteCount')

In [10]:
# Ramkę wynikową łączymy z tą, w której w grupach sumowaliśmy wartości FavoriteCount wg równości
# zmiennej OwnerUserId
tmp4 = tmp3.merge(tmp1, on = 'OwnerUserId')

In [11]:
# Tak uzyskaną ramkę danych łączymy z kolumnami 'DisplayName', 'Age', 'Location', 'Id' z ramki Users.
# Kryterium scalenia to równość "Id" z ramki Users i "OwnerUserId" z ramki uzyskanej powyżej
# Następnie sortujemy wartości malejąco względem zmiennej 'FavoriteTotal', 
# nadajemy nowe indeksy (od 0), 
# wybieramy pierwsze 10 rekordów oraz ustawiamy kolumny w odpowiedniej kolejności, zmieniając przy tym nazwę 
# kolumny "Title" na "MostFavoriteQuestion"
res_1 = Users[['DisplayName', 'Age', 'Location', 'Id']].merge(tmp4,
        left_on = 'Id', right_on = 'OwnerUserId').sort_values( 
    'FavoriteTotal', ascending = False).reset_index().iloc[:10,][
    ['DisplayName', 'Age', 'Location', 'FavoriteTotal', 'Title', 'MostFavoriteQuestionLikes']].rename(
    columns = {"Title":"MostFavoriteQuestion"})

## 1.2. Sprawdzenie poprawności otrzymanych rezultatów.

### 1.2.1. Sprawdzenie klasy otrzymanego obiektu.

In [12]:
res_1.__class__ # Upewniamy się, że wynik to obiekt DataFrame a nie Slice.

pandas.core.frame.DataFrame

### 1.2.2. Sprawdzenie poprawności danych wynikowych.

In [13]:
res_1.equals(sql_1)  # Wszystko się zgadza, ale czy na pewno?

True

In [14]:
np.equal(res_1, sql_1)

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,True,True,True,True,True,True
1,True,False,True,True,True,True
2,True,False,False,True,True,True
3,True,True,True,True,True,True
4,True,False,False,True,True,True
5,True,False,True,True,True,True
6,True,True,True,True,True,True
7,True,True,True,True,True,True
8,True,False,False,True,True,True
9,True,False,True,True,True,True


In [15]:
# Zwróćmy uwagę na miejsca, w których występują różnicę między ramką res_1, sql_1. Otóż występują one wyłącznie
# w miejscach, w których mamy do czynienia z brakami danych. Co ciekawe funkcja np.equals() zwraca 'False' nie tylko
# wtedy, gdy porównuje obiekty None i NaN, ale również porównując NaN z NaN. Dlatego też nie zdecydowałam się 
# na zamianę obiektów NaN na None ani odwrotnie.

# Uwaga!

In [16]:
# Zwróćmy uwagę, że wszędzie gdzie nie występują braki danych numpy.equal() zwraca True. 
# Zatem otrzymane w rezultacie dane są zgodne.

# Zadanie 2.

## 2.1. Jaki rezultat mamy osiągnąć?
### W tym zadaniu chcemy uzyskać informacje o postach, które w poszczególnych latach miały najwięcej głosów typu '2'.
Dane, które nas interesują to Id najczęściej lajkowanych postów, ich treść oraz ogólna liczba pozytywnych reakcji na wpisy ich autorów. 

In [17]:
sql_2 = pd.read_sql_query(
    """
    SELECT
            Posts.ID,
            Posts.Title,
            Posts2.PositiveAnswerCount
        FROM Posts
        JOIN (
            SELECT
                Posts.ParentID,
                COUNT(*) AS PositiveAnswerCount
            FROM Posts
            WHERE Posts.PostTypeID=2 AND Posts.Score>0
            GROUP BY Posts.ParentID
                        ) 
            AS Posts2
            ON Posts.ID=Posts2.ParentID
    ORDER BY Posts2.PositiveAnswerCount DESC
    LIMIT 10
    """, conn)

In [18]:
# Wybieramy posty, dla których wartości zmiennych spełniają następujące warunki: 
# 'PostTypeId' wynosi 2 i 'Score' jest dodatnia,
# otrzymaną w rezultacie ramkę grupujemy po zmiennej 'ParentId', następnie za pomocą funkcji size()
# otrzymujemy liczność poszczególnych grup, czyli liczbę pozytywnych rekacji na wszystkie wpisy autorów  
# rozważanych postów

Posts2 =  Posts.loc[(Posts.PostTypeId == 2) & (Posts.Score > 0), ['ParentId']].groupby('ParentId').size(
            ).reset_index(name='PositiveAnswerCount')

# W ostatnich krokach resetujemy indeksy i nazywamy kolumnę przechowującą rozmiary 
# grup jako 'PositiveAnswerCount'. Otrzymaną w ten sposób ramkę nazywamy jako 'Posts2'

In [19]:
# Ramkę wynikową łączymy z kolumnami 'Id' i 'Title' z ramki Posts wg następującego kryterium:
# Posts.ID=Posts2.ParentID. Aby uniknąć duplikatów używamy funkcji drop_duplicates, a w celu pominięcia
# kolumny ParentId, której nie chcemy mieć w finalnej ramce stosujemy drop na niechcianej kolumnie

res_2 = Posts[['Id', 'Title']].merge(Posts2, left_on = 'Id', right_on = 'ParentId').drop_duplicates(
        ).drop(columns = 'ParentId')

# Następnie, jak niżej, dane sortujemy malejąco względem zmiennej PositiveAnswerCount i wybieramy
# pierwsze 10 wierszy

# Dodatkowa ramka - na potrzeby testów
df3 = res_2.sort_values(['PositiveAnswerCount'], ascending = False).reset_index().drop(columns = 'index'
                                                                                      ).iloc[:10,]

res_2 = res_2.sort_values(['PositiveAnswerCount'], ascending = False).reset_index().drop(columns = 'index'
                                                                                      ).iloc[:10,]

## 2.2. Sprawdzenie zgodności otrzymanych danych.

### 2.2.1. Sprawdzenie klasy otrzymanego obiektu.

In [20]:
res_2.__class__

pandas.core.frame.DataFrame

### 2.2.2. Sprawdzenie poprawności danych wynikowych.

In [21]:
np.equal(sql_2, res_2)

,Id,Title,PositiveAnswerCount
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,False,False,True
5,False,False,True
6,True,True,True
7,False,False,True
8,False,False,True
9,False,False,True


# UWAGA!

In [23]:
# Zwrócone wartości False spowodowane są różnicami w permutacji wierszy porównywanych ramek. 
# Z tego również powodu w każdej ramce znajduje się jeden wiersz, który nie występuje w drugiej. Dlaczego? 
# Otóż ostatnim krokiem w procesie przetwarzania danych był wybór 10 pierwszych wierszy. Okazuje się, że w 
# w przypadku wierszy 7-11 mamy do czynienia z remisem. Ze względu na to, że wiersze o takiej samej wartości
# 'Score' zostają zwrócone w ramkach w innej kolejności, to wiersz który w ramce sql_2 znalazł się na 10 miejscu
# w ramce res_2 znalazłby się na jedenastym. A ponieważ wybieraliśmy pierwsze 10 rekorów, to ostatecznie nie znalazł
# się w niej wcale. Nie mniej jednak zwrócone dane są poprawne i mam nadzieję, że te subtelne różnice nie 
# wpłyną negatywnie na ocenę. 
# Aby udowodnić, że ramki posiadają takie same dane zmodyfikujmy nieco polecenie i spójrzmy jak wyglądałyby 
# gdybyśmy wzięli pierwsze 11 wierszy dla każdej. 

In [22]:
df_sql = pd.read_sql_query(
    """
    SELECT
            Posts.ID,
            Posts.Title,
            Posts2.PositiveAnswerCount
        FROM Posts
        JOIN (
            SELECT
                Posts.ParentID,
                COUNT(*) AS PositiveAnswerCount
            FROM Posts
            WHERE Posts.PostTypeID=2 AND Posts.Score>0
            GROUP BY Posts.ParentID
                        ) 
            AS Posts2
            ON Posts.ID=Posts2.ParentID
    ORDER BY Posts2.PositiveAnswerCount DESC
    LIMIT 11
    """, conn)

In [23]:
df_sql

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


In [24]:
df3

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
5,3220,Why would you wrap your luggage in plastic?,19
6,30656,Long-life SIM cards in Europe,18
7,80329,How to cope with too slow Wi-Fi at hotel?,17
8,7663,Am I expected to tip wait staff in Europe?,17
9,59128,"Is there a way to prevent ""looking like a tour...",17


In [86]:
# Zwróćmy uwagę, że są to dokładnie te same dane, tylko że występują w innej kolejności. 
# Dla pewności, że nic mi nie umnkęło sprawdziłam też typy zwracanych danych. 

In [25]:
df3.dtypes

Id                      int64
Title                  object
PositiveAnswerCount     int64
dtype: object

In [26]:
df_sql.dtypes

Id                      int64
Title                  object
PositiveAnswerCount     int64
dtype: object

In [30]:
# One również są takie same. 

# Zadanie 3.

## 3.1. Jaki rezultat mamy osiągnąć?
### W tym zadaniu chcemy uzyskać informacje o postach, które w kolejnyvh latach spotykały się z największą liczbą pozytywnnych reakcji. 
Dane jakie chcemy uzyskać to tytułu postów, daty ich utworzenia i maksymalne liczby pozytywnych reakcji uzyskanych w skali rocznej.

In [27]:
sql_3 = pd.read_sql_query(
    """
    SELECT
        Posts.Title,
        UpVotesPerYear.Year,
        MAX(UpVotesPerYear.Count) AS Count
    FROM (
        SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
        ) AS UpVotesPerYear
    JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
    WHERE Posts.PostTypeId=1
    GROUP BY Year
    """, conn)

In [28]:
# Z ramki Votes wybieramy wiersze, dla których zmienna 'VoteTypeId' przyjmuje wartość 2 oraz kolumny: 'PostId' i
# 'CreationDate'.
UpVotesPerYear = Votes[Votes.VoteTypeId == 2][['PostId', 'CreationDate']]

# Następnie z daty ekstrahujemy rok.
# W tym celu zamieniamy kolumnę CreationDate na listę i tworzymy listę pomocniczą Date,
# w której będziemy zamieniać pełne daty na same lata.
Date = UpVotesPerYear.CreationDate.values.tolist()
n = len(Date)

for i in range(n):
    Year = Date[i].split("-")[0]
    Date[i] = Year
    
# Listę Date z zamienionymi wartościami umieszczamy w ramce
UpVotesPerYear['Year'] = Date

# Opuszczamy niepotrzebną już kolumnę CreationDate, grupujemy dane po zmiennych 'PostId',
# 'Year' i za pomocą funkcji size() określamy liczności otrzymanych grup.
UpVotesPerYear = UpVotesPerYear.drop(columns = 'CreationDate').groupby(
    ['PostId', 'Year']).size().reset_index(name='Count')

In [29]:
# Z ramki Posts wybieramy wiersze 'PostTypeId'==2 i kolumny 'Id', 'Title'. Kolumna 'Id' jest nam potrzebna tylko
# do konkatenacji. Otrzymane dane łączymy z ramką UpVotesPerYear wg kryterium Posts.Id=UpVotesPerYear.PostId. 
# Następnie grupujemy dane po zmiennej Year.
df = Posts[Posts.PostTypeId == 1][['Id', 'Title']].merge(UpVotesPerYear, left_on = 'Id', right_on = 'PostId').drop(
    columns = ['Id', 'PostId'])
tmp = df.groupby('Year')
# Na otrzymanych grupach agregujemy funkcję max()
res_3 = pd.DataFrame([{'Count': ramka.Count.max(skipna = True),
                      'Year': grupa} 
                       for (grupa, ramka) in tmp])
# I łączymy ramkę df z tą samą ramką po agregacji max
# Opuszczamy kolumnę 'Year_y', zmieniamy nazwę kolumny 'Year_x' na 'Year', resetujemy
# indeksy i zmieniamy kolejność kolumn.
res_3 = res_3.merge(df, on = 'Count')
res_3 = res_3[res_3.Year_x == res_3.Year_y].drop(columns = 'Year_y').rename(
    columns = {"Year_x":"Year"}).reset_index().drop(columns = 'index')[['Title', 'Year', 'Count']]

## 3.2. Sprawdzenie zgodności otrzymanych danych.

### 3.2.1. Sprawdzenie klasy otrzymanego obiektu.

In [30]:
res_3.__class__

pandas.core.frame.DataFrame

### 3.2.2. Sprawdzenie poprawności danych wynikowych.

In [31]:
res_3.equals(sql_3)  # Wszystko się zgadza, ale czy na pewno?

True

In [32]:
np.equal(sql_3, res_3) # Tak :)

,Title,Year,Count
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
5,True,True,True
6,True,True,True


# Zadanie 4.

## 4.1. Jaki rezultat mamy osiągnąć?
### Wybieramy te pytania, do których najlepiej ocenione odpowiedzi miały ponad 50 głosów więcej niż te pytania. 
Dane, na których nam zależy to Id i tytuł pytania, maksymalna ocena odpowiedzi, ocena pytania i różnica tych ocen.

In [33]:
sql_4 = pd.read_sql_query(
    """
    SELECT
        Questions.Id,
        Questions.Title,
        BestAnswers.MaxScore,
        Posts.Score AS AcceptedScore,
        BestAnswers.MaxScore-Posts.Score AS Difference
    FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
        ) AS BestAnswers
    JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
        ) AS Questions
        ON Questions.Id=BestAnswers.ParentId
    JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
    WHERE Difference>50
    ORDER BY Difference DESC
    """, conn)

In [34]:
# Z ramki Posts wybieramy Posty typu '2' i grupujemy je względem zmiennej 'ParentId'.
tmp = Posts[Posts.PostTypeId == 2].groupby('ParentId')
# Na tak otrzymanych danych agregujemy funkcję max() nazywając odpowiednio kolumny.
tmp2 = pd.DataFrame([{'MaxScore': ramka.Score.max(skipna = True),
                      'ParentId': grupa} 
                       for (grupa, ramka) in tmp])
# Otrzymaną ramkę łączymy z wybranymi kolumnami ramki Posts. Konkatenacji dokonujemy po dwóch zmiennych,
# aby mieć pewność, że ramki zostaną właściwie połączone. Ramkę wynikową przypisujemy zmiennej 'BestAnswers'.
BestAnswers = tmp2.merge(Posts[['Id', 'ParentId', 'Score']], left_on = ['MaxScore', 'ParentId'],
                         right_on = ['Score', 'ParentId'])[['Id', 'ParentId', 'MaxScore']]

In [35]:
# Zmiennej 'Questions' przypisujemy posty typu 1.
Questions = Posts[Posts.PostTypeId == 1]
# Łączymy wybrane kolumny z ramki Questions z ramką BestAnswers wg równości Id i ParentId
tmp3 = Questions[['Id', 'Title', 'AcceptedAnswerId']].merge(BestAnswers, left_on = 'Id', right_on = 'ParentId')
# następnie dokonujemy konkatenacji ramki otrzymanej wyżej z wybranymi kolumnami ramki Posts
res_4 = tmp3.merge(Posts[['Score', 'Id']], left_on = 'AcceptedAnswerId', right_on = 'Id')
# dla każdego wiersza obliczamy różnicę między maksymalną wartością Score a rzeczywistą wartością.
Difference = res_4.MaxScore-res_4.Score
# Kolumnę wynikową 'doklejamy' do przetwarzanej ramki
res_4 = pd.concat([res_4.reset_index(drop=True), Difference.rename('Difference')], axis=1)

In [36]:
# Wybieramy te wiersze, dla których różnica jest większa od 50, sortujemy dane malejąco względem tej różnicy 
# i wybieramy potrzebne kolumny. 
res_4 = res_4[res_4.Difference > 50].sort_values('Difference', ascending = False)[["Id_x", "Title", "MaxScore",
                                                                                   "Score", "Difference"]]
# Uwaga! Możemy wybrać dowolne Id, bo Id_x == Id_y

In [37]:
# Zmieniamy nazwy kolumn i resetujemy indeks i wybieramy potrzebne kolumny. 
res_4 = res_4.rename({'Id_x': 'Id', 'Score': 'AcceptedScore'}, axis = 1).reset_index()[["Id", "Title", "MaxScore",
                                                                                   "AcceptedScore", "Difference"]]

## 4.2. Sprawdzenie zgodności otrzymanych danych.

### 4.2.1. Sprawdzenie klasy otrzymanego obiektu.

In [38]:
res_4.__class__

pandas.core.frame.DataFrame

### 4.2.2. Sprawdzenie poprawności danych wynikowych.

In [39]:
res_4.equals(sql_4)  # Wszystko się zgadza, ale czy na pewno?

True

In [40]:
np.equal(sql_4, res_4) # Bez zarzutu ;)

,Id,Title,MaxScore,AcceptedScore,Difference
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
5,True,True,True,True,True
6,True,True,True,True,True
7,True,True,True,True,True


# Zadanie 5.

## 5.1. Jaki rezultat mamy osiągnąć?
### W tym zadaniu chcemy uzyskać informacje o 10 postach, które mają najwięcej komentarzy.
Wartość _Score_ dla komentarza oblicza się odejmując od liczby pozytywnych reakcji liczbę tych negatywnych.
Wartość CommentsTotalScore, obliczymy samodzielnie wg algorytmu opisanego w komentarzach poniżej. 

In [41]:
sql_5 = pd.read_sql_query(
    """
    SELECT
        Posts.Title,
        CmtTotScr.CommentsTotalScore
    FROM (
        SELECT
            PostID,
            UserID,
            SUM(Score) AS CommentsTotalScore
        FROM Comments
        GROUP BY PostID, UserID
        ) AS CmtTotScr
        JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
        WHERE Posts.PostTypeId=1
        ORDER BY CmtTotScr.CommentsTotalScore DESC
        LIMIT 10
            """, conn)

In [42]:
# W celu zminimalizowania ilości przetwarzanych danych z ramki Comments wybieramy niezbędne kolumny:
# 'Score', 'PostId', 'UserId'. Następnie grupujemy dane względem zmiennych 'PostId' i 'UserId'.
tmp = Comments[['Score', 'PostId', 'UserId']].groupby(['PostId', 'UserId'])

In [43]:
# Mając takie grupy, agregujemy sumę na kolumnie 'Score'.
CmtTotScr =  pd.DataFrame([{'CommentsTotalScore' : ramka.Score.sum(skipna = True),
                            'PostId': grupa[0],
                            'UserId': grupa[1]} 
                        for (grupa, ramka) in tmp])

In [44]:
# Z ramki Posts wybieramy kolumnę 'Title', potrzebną do ramki wynikowej, 'PostTypeId' potrzebne do późniejszego
# filtrowania danch oraz kolumny: 'Id' i 'OwnerUserId', niezbędne do konkatenacji z ramką CmtTotScr. 
# Następnie łączymy te ramki zgodnie z warunkami: Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
res_5 = Posts[['Title', 'PostTypeId', 'Id', 'OwnerUserId']].merge(CmtTotScr, left_on = ['Id', 'OwnerUserId'],
                                                               right_on = ['PostId', "UserId"])
# Wybieramy te wiersze, dla których spełniony jest warunek: PostTypeId=1 i wybieramy kolumny 'Title' i 
# 'CommentsTotalScore' potrzebne do ramki wynikowej. 
res_5 = res_5[res_5.PostTypeId == 1][['Title', 'CommentsTotalScore']]
# Na koniec sortujemy dane malejąco wzgledem zmiennej 'CommentsTotalScore', zerujemy numery wierszy
# i wybieramy pierwsze 10 wierszy.
res_5 = res_5.sort_values(['CommentsTotalScore'],ascending = False).reset_index(drop = True).iloc[:10,]

## 5.2. Sprawdzenie zgodności otrzymanych danych.

### 5.2.1. Sprawdzenie klasy otrzymanego obiektu.

In [45]:
res_5.__class__

pandas.core.frame.DataFrame

### 5.2.2. Sprawdzenie poprawności danych wynikowych.

In [46]:
np.equal(sql_5, res_5)

,Title,CommentsTotalScore
0,True,True
1,True,True
2,True,True
3,False,True
4,False,True
5,False,True
6,True,True
7,True,True
8,False,True
9,False,True


# Uwaga!

In [49]:
# Wszelkie niezgodności wynikają z różnych permutacji wierszy w porównywanych ramkach. Spójrzmy poniżej i sami 
# porównajmy. Wszystkie różnice występują wówczas, gdy mamy sytuację remisu. 

In [47]:
sql_5

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


In [48]:
res_5

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,Can I have a watermelon in hand luggage?,25
4,How does President Trump's travel ban affect n...,25
5,What's the longest scheduled public bus ride i...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,India just demonetized all Rs 500 & 1000 notes...,20
9,Legalities and safety concerns of visiting pro...,20


# Zadanie 6. 

## 6.1. Jaki rezultat mamy osiągnąć?
### Wybieramy użytkowników, którzy mają odznakę klasy '1', którą dotychczas przyznano co najmniej 2 razy i co najwyżej 10 razy.
Dane o użytkownikach, które chcemy wyekstrahować to: Id, nazwa, wartość zmiennej 'Reputation', wiek oraz lokalizacja.

In [50]:
sql_6 = pd.read_sql_query("""
                SELECT DISTINCT
                      Users.Id,
                      Users.DisplayName,
                      Users.Reputation,
                      Users.Age,
                      Users.Location
                  FROM (
                          SELECT
                            Name, UserID
                          FROM Badges
                          WHERE Name IN (
                            SELECT
                              Name
                            FROM Badges
                            WHERE Class=1
                            GROUP BY Name
                            HAVING COUNT(*) BETWEEN 2 AND 10
                      )
                          AND Class=1
                    ) AS ValuableBadges
                  JOIN Users ON ValuableBadges.UserId=Users.Id""", conn)

In [51]:
# Z ramki Badges(odznaki) wybieramy wiersze, których klasa to 1. Otrzymane dane grupujemy po zmiennej 'Name'.
# Za pomocą size() określamy liczności poszczególnych grup. Liczności te umieszczamy w nowej kolumnie 'Count'. 
tmp = Badges[Badges.Class == 1].groupby('Name').size().reset_index(name='Count')
# Wybieramy te nazwy, dla których liczności grup zawierają się w przedziale obustronnie domkniętym [2, 10]
tmp = tmp[(tmp.Count >= 2) & (tmp.Count <= 10)][['Name']]
# Kolumnę names zamieniamy na listę, aby móc użyć jej jako argument funkcji isin()
tmp = tmp['Name'].tolist()
# Z ramki Badges wybieramy te wiersze, których nazwy występują w wyżej otrzymanej ramce.
ValuableBadges = Badges[Badges.Name.isin(tmp)]
# Z tak otrzymanego zasobu danych wybieramy wiersze z odznakami klasy 1 i kolumny 'Name', 'UserId'.
ValuableBadges = ValuableBadges[ValuableBadges.Class == 1][['Name', 'UserId']]
# Następnie łączymy ramkę ValuableBadges z ramką Users wg kryterium ValuableBadges.UserId=Users.Id 
# i wybieramy pożądane kolumny. 
res_6 = ValuableBadges.merge(Users, left_on = 'UserId', right_on = "Id")[['Id', 'DisplayName','Reputation', 'Age','Location']]
res_6 = res_6.drop_duplicates().reset_index(drop = True)

## 6.2. Sprawdzenie zgodności otrzymanych danych.

### 6.2.1. Sprawdzenie klasy otrzymanego obiektu.

In [52]:
res_6.__class__

pandas.core.frame.DataFrame

### 6.2.2. Sprawdzenie poprawności danych wynikowych.

In [53]:
res_6.equals(sql_6) # Wszystko się zgadza, ale czy na pewno?

True

In [54]:
np.equal(res_6, sql_6)

,Id,DisplayName,Reputation,Age,Location
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,False,True
4,True,True,True,True,True
5,True,True,True,False,False
6,True,True,True,False,False
7,True,True,True,False,True
8,True,True,True,False,True
9,True,True,True,True,True


In [59]:
# Wszystkie wartości 'False' są zwrócone w miejscach, gdzie występują braki danych. 
# Wynika to z tego, o czym pisałam przy okazji pierwszego zadania. A mianowicie z braku umiejętności porównywania
# wyrazów NaN. Dlatego też, tak jak w pierwszym zadaniu, nie zdecydowałam się na zamianę obiektów NaN na None, gdyż
# ostatecznie porównanie i tak w tych miejscach zwróciłoby 'False'.

# Zadanie 7. 

## 7.1. Jaki rezultat mamy osiągnąć?
### W tym zadaniu chcemy dowiedzieć się jakie posty miały najwięcej głosów przed rokiem 2016 i 0 w latach 2016-2017.

In [55]:
sql_7 = pd.read_sql_query("""
                SELECT
                      Posts.Title,
                      VotesByAge2.OldVotes
                  FROM Posts
                  JOIN (
                      SELECT
                            PostId,
                            MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
                            MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
                            SUM(Total) AS Votes
                      FROM (
                          SELECT
                              PostId,
                              CASE STRFTIME('%Y', CreationDate)
                                  WHEN '2017' THEN 'new'
                                  WHEN '2016' THEN 'new'
                                  ELSE 'old'
                                  END VoteDate,
                              COUNT(*) AS Total
                          FROM Votes
                          WHERE VoteTypeId=2
                          GROUP BY PostId, VoteDate
                      ) AS VotesByAge
                      GROUP BY VotesByAge.PostId
                      HAVING NewVotes=0
                    ) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
                    WHERE Posts.PostTypeId=1
                    ORDER BY VotesByAge2.OldVotes DESC
                    LIMIT 10
""", conn)

In [56]:
# Z ramki Votes wybieramy głosy typu drugiego i kolumny PostId i CreationDate
df1 = Votes[Votes.VoteTypeId == 2][['PostId', 'CreationDate']]
# Następnie z daty ekstrahujemy rok.
# W tym celu zamieniamy kolumnę CreationDate na listę i tworzymy listę pomocniczą VoteDate,
# w której będziemy zamieniać pełne daty na same lata i odpowiednio je etykietować.
VoteDate = df1.CreationDate.values.tolist()
n = len(VoteDate)
# Jeśli głos został uwtorzony przed rokiem 2016 to etykietujemy go jako 'old', a jeśli w latach 2016-2017, jako 'new'

for i in range(n):
    Year = VoteDate[i].split("-")[0]
    if (Year == '2017' or Year == '2016'):
        VoteDate[i] = 'new'
    else:
        VoteDate[i] = 'old'

# Listę VoteDate z zamienionymi wartościami umieszczamy w ramce
df1['VoteDate'] = VoteDate

In [57]:
# Z ramki wybieramy kolumny 'PostId', 'VoteDate'. Pozostałe nie będą nam już potrzebne do dalszej pracy. 
# Grupujemy po zmiennych znajdujących się w wybranych kolumnach i za pomocą funkcji size() określamy liczności
# poszczególnych grup. Otrzymane wyniki umieszczamy w nowej kolumnie o nazwie 'Total'. 
VotesByAge = df1[['PostId', 'VoteDate']].groupby(['PostId', 'VoteDate']).size(
            ).reset_index(name='Total')

In [58]:
# Kolumna VoteDate jest wspólna. Występują w niej zarówno stare, jak i nowe głosy.
# To co będziemy teraz robić, to rozdzielenie kolumny 'VoteDate' na dwie. 
# Jedna z nich będzie przechowywać ilościowe informacje o nowych głosach a druga o starych. 
# Aby to zrobić zamieniamy kolumny VoteDate oraz Total na listy. W celu uniknięcia pomyłki listy pomocniczne 
# nazywamy nazwami kolumn.
VoteDate = VotesByAge.VoteDate.values.tolist()
Total = VotesByAge.Total.values.tolist()
n = len(VoteDate)
# Tworzymy nowe listy NewVotes i OldVotes, które staną się naszymi nowymi kolumnami.
NewVotes = [0 for i in range(n)]
OldVotes = [0 for i in range(n)]
# A następnie za pomocą pętli przechodzimy po liście VoteDate i w odpowiedniej liście Old/New Votes zapisujemy
# ilości poszczególnych głosów.
for i in range(n):
    if VoteDate[i] == 'new':
        NewVotes[i] = Total[i]
    else:
        OldVotes[i] = Total[i]

In [59]:
# Tworzymy nową ramkę złożoną z kolumn 'PostId', 'Total' ramki VotesByAge.
df = VotesByAge[['PostId', 'Total']]
# A następnie dokonujemy na niej konkatenacji z otrzymanymi wyżej listami Old/New Votes
df['NewVotes'] = NewVotes
df['OldVotes'] = OldVotes

In [60]:
# Tak otrzymaną ramkę grupujemy po zmiennej 'PostId'
tmp = df.groupby('PostId')
# A następnie na pogrupowanych danych agregujemy funkcje: sum i max.
# Sumujemy Total a maksymalne wartości wyciągamy dla kolumn Old/New Votes.
# Tak powstałąm ramkę danych opatrujemy nazwą VotesByAge2
VotesByAge2 =  pd.DataFrame([{'Votes' : ramka.Total.sum(skipna = True),
                       'NewVotes' : ramka.NewVotes.max(skipna = True),
                       'OldVotes' : ramka.OldVotes.max(skipna = True),
                       'PostId': grupa} 
                        for (grupa, ramka) in tmp])

In [61]:
# Z utworzonej przed chwilą ramki ekstrachujemy głosy, które nie miały nowych głosów (VotesByAge2.NewVotes == 0)
VotesByAge2 = VotesByAge2[VotesByAge2.NewVotes == 0]
# A następnie łączymy ją z ramką Posts.
res_7 = Posts.merge(VotesByAge2, left_on = 'Id', right_on = 'PostId')
# Ramka df7, to ramka dodatkowa na potrzeby testów
df7 = res_7[res_7.PostTypeId == 1].sort_values('OldVotes', ascending = False)[['Title', 'OldVotes']].reset_index().iloc[:12,].drop(columns = 'index')
# Wybieramy wiersze, dla których typ postu to: 1. Sortujemy wartości malejąco po zmiennej 'OldVotes' a następnie
# wybieramy pierwsze 10 rekordów.
# Ramka df7, to ramka dodatkowa na potrzeby testów
df7 = res_7[res_7.PostTypeId == 1].sort_values('OldVotes', ascending = False)[['Title', 'OldVotes']].reset_index().iloc[:12,].drop(columns = 'index')
res_7 = res_7[res_7.PostTypeId == 1].sort_values('OldVotes', ascending = False)[['Title', 'OldVotes']].reset_index().iloc[:10,].drop(columns = 'index')

## 7.2. Sprawdzenie zgodności otrzymanych danych.

### 7.2.1. Sprawdzenie klasy otrzymanego obiektu.

In [62]:
res_7.__class__

pandas.core.frame.DataFrame

### 7.2.2. Sprawdzenie poprawności danych wynikowych.

In [63]:
np.equal(res_7, sql_7)

,Title,OldVotes
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,False,True
7,False,True
8,False,True
9,True,True


In [ ]:
# W tym zadaniu powtarza nam się historia z zadania 2. 
# Zwróćmy uwagę jak w powyższej tabeli ładnie widać, że wiersze, które się różnią w ramkach mają takie same wartości
# zmiennej, względem której sortowaliśmy. 
# Aby udowodnić, że ramki posiadają takie same dane zmodyfikujmy nieco polecenie i spójrzmy jak wyglądałyby 
# gdybyśmy wzięli pierwsze 12 wierszy dla każdej. 

In [64]:
df_sql_7 = pd.read_sql_query("""
                SELECT
                      Posts.Title,
                      VotesByAge2.OldVotes
                  FROM Posts
                  JOIN (
                      SELECT
                            PostId,
                            MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
                            MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
                            SUM(Total) AS Votes
                      FROM (
                          SELECT
                              PostId,
                              CASE STRFTIME('%Y', CreationDate)
                                  WHEN '2017' THEN 'new'
                                  WHEN '2016' THEN 'new'
                                  ELSE 'old'
                                  END VoteDate,
                              COUNT(*) AS Total
                          FROM Votes
                          WHERE VoteTypeId=2
                          GROUP BY PostId, VoteDate
                      ) AS VotesByAge
                      GROUP BY VotesByAge.PostId
                      HAVING NewVotes=0
                    ) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
                    WHERE Posts.PostTypeId=1
                    ORDER BY VotesByAge2.OldVotes DESC
                    LIMIT 12
""", conn)
df_sql_7

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


In [65]:
df7

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,Should I avoid overnight flights from Dubai to...,23
7,"Can a visitor who is ""of age"" in their country...",23
8,What is the business model of commercial free ...,23
9,Carrying medicines internationally for a friend,23


### Dla pewności sprawdźmy typy danych zwracanych w ramce wynikowej. 

In [66]:
df7.dtypes

Title       object
OldVotes     int64
dtype: object

In [67]:
df_sql_7.dtypes

Title       object
OldVotes     int64
dtype: object

In [69]:
# Tutaj wszystko się zgadza. Zatem wszelkie nieprawidłowości muszą wynikać z różnych permutacji wierszy
# o takich samych wartościach zmiennej, względem której sortowaliśmy.

# Drobna uwaga :)

Mam nadzieję, że tego typu ocena poprawności wyników jest wystarczająca. Nawiązując do naszej rozmowy postanowiłam po prostu zakomentować wszelkie nierprawidłowości i mam nadzieję, że udało mi się to zrobić w sposób wyczerpujący i tym samym przekonujący. 
Wszelkie adnotacje pt. 'Jaki rezultat mamy osiągnąć' robiłam z myślą o opisie dla laików, natomiast kolejne kroki rozwiązań poszczególnych zadań są umieszczone w komentarzach wewnątrz kodu. 
Zgodnie z treścią w przypadku każdego zadania, upewniłam się, że zwracane wyniki są ze sobą tożsame (ewentualnie z dokładnością do permutacji wierszy wynikowych ramek danych).
Sprawdziłam też, że w każdym przypadku wynik jest klasy DataFrame a nie Series.
Wewnątrz dokumentu pozostawiam oryginalne ścieżki do plików, które zczytywałam na potrzeby realizacji projektu. 
P.S. Mam nadzieję, że tym razem zrehabilitowałam się za brak komentarzy w pracy domowej nr 1 :).